In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import PIL
from PIL import Image
from torch.utils import data as D
from  torch.utils.data.sampler import SubsetRandomSampler
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [11]:

batch_size = 64
validation_rate = 0.1
random_seed = 10
lr = 1e-1
epoches = 300

transform_train = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616))
])
transform_valid = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])
transform_test = transforms.Compose([
  transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
validset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_valid)
testset=torchvision.datasets.CIFAR100(root='./data',train=False,download=True,transform=transfrom_test)


Files already downloaded and verified
Files already downloaded and verified



 98%|█████████▊| 166379520/170498071 [01:12<00:01, 2306482.55it/s] 
170500096it [00:34, 4873448.31it/s] 

Extracting ./data\cifar-100-python.tar.gz to ./data


In [12]:
from torch.utils.data import DataLoader
num_trains = len(trainset)
indice = list(range(num_trains))
spilt = int(np.floor(validation_rate * num_trains))

np.random.seed(random_seed)
np.random.shuffle(indice)

train_idx, valid_idx = indice[spilt:], indice[:spilt]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_loader = DataLoader(trainset, batch_size=batch_size, sampler=train_sampler, num_workers=0)
valid_loader = DataLoader(validset, batch_size=batch_size, sampler=valid_sampler, num_workers=0)
test_loader = DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=0)

classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [16]:
class conv_model(nn.Module):
  def __init__(self, nin, nout, kernel_size, stride, padding, bias=False):
    super(conv, self).__init__()
    self.bn = nn.BatchNorm2d(nin)
    self.relu = nn.ReLU()
    self.conv = nn.Conv2d(nin, nout, kernel_size=kernel_size, stride=stride, padding=padding)
    
  def forward(self, x):
    out = self.batch_norm(x)
    out = self.relu(out)
    out = self.conv(out)
    return out 

In [ ]:

class bottlenect(nn.Module):
  def __init__(self, nin, growth_rate, drop_rate=0.2):
    super(bottlenect, self).__init__()
    self.add_module('conv1x1', conv_model(nin=nin, nout=growth_rate * 4, kernel_size=1, stride=1, padding=0, bias=False))
    self.add_module('con3x3', conv_model(nin=growth_rate * 4, nout=growth_rate, kernel_size=3, stride=1, padding=1, bias=False))
    self.drop_rate = drop_rate
    
  def forward(self, x):
    bottlenect_output = super(bottlenect, self).forward(x)
    if self.drop_rate > 0:
      bottlenect_output = F.dropout(bottlenect_output, p=self.drop_rate, training=self.training)  # 트레이닝은 없는 값인데 문제가 되지않나용ㅇ?
      
    bottlenect_output = torch.cat((x, bottlenect_output), 1) # 1이 디멘션을 얘기한다는데 concat 을 할때 하지만 기본값이 (3,64,64)라고 하면 3이 채널값인데 0을 넣어주는게 맞지않나 
    
    return bottlenect_output


    

In [17]:

class transition_layer(nn.Sequential):
  def __init__(self, nin, theta=.5):
    super(transition_layer, self).__init__()
    self.add_module('conv_1x1', conv_model(nin=nin, nout=int(nin * theta), kernel_size=1, stride=1, padding=0, bias=False))
    self.add_module('avg_pool_2x2', nn.AvgPool2d(kernel_size=2, stride=2, padding=0))
    
class denseblock(nn.Sequential):
  def __init__(self, nin, nout,nin_bottlenect, growth_rate, drop_rate=0.2):
    super(denseblock, self).__init__()
    
    for i in range(bottlenect):
      nin_bottlenect = nin + growth_rate * i
      self.add_module('bottlenect_%d %i', bottlenect(nin=nin_bottlenect, growth_rate=growth_rate, drop_rate=drop_rate))
      

In [ ]:

class Densenet(nn.Module):
  def __init__(self, growth_rate=12, num_layers=100, theta=.5, drop_rate=0.2, num_classes=10):
    super(Densenet, self).__init__()
    assert (num_layers - 4) % 6 == 0
    
    #
    num_bottleneck_layers = (num_layers - 4) // 6
    self.dense_init = nn.Con2d(3, growth_rate * 2, kernel_size=3, stride=1, padding=1, bias=True)
    self.dense_block_1 = denseblock(nin=growth_rate * 2, num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)
    nin_transition_layer_1 = growth_rate * 2 + growth_rate * num_bottleneck_layers
    self.transition_layer_1 = transition_layer(nin=nin_transition_layer_1, theta=theta)
    self.dense_block_2 = denseblock(nin=int(nin_transition_layer_1 * theta), num_bottleneck_layers=num_bottleneck_layers)
    
    